# Resolvendo Pacman
_The definition of the problem, the solution, and the results obtained must be presented in a report created as a
Jupyter notebook. Please, make sure you put the graphs, tables, comparisons, and critical analysis in the notebook.
The report should clearly indicate what the contribution of each team member was._

O objetivo deste trabalho é de resolver o jogo Pacman, de forma estática, utilizando os algoritmos de busca aprendidos nas aulas de MC906/MO416.

## Algoritmos utilizados
1. Breadth first search
2. Depth first search
3. Greedy Best First search
4. A*
5. Hill climbing search
# **explicar a escolha dos métodos depois**

## Heurísticas

As Heurísticas elaboradas foram determinantes, principalmente para os Informed Search Algorithms, onde seu resultado h(n) é contabilizado juntamente do path cost   g(n) para o agente decidir o próximo passo a ser avaliado. Podendo ou não indicar a melhor solução.

Para o problema do Pacman nossas heurísticas estão voltadas para achar o menor caminho possível para nosso agente chegar ao *goal*, não foram parametrizados as moedas espalhadas pelo mapa.

Com uma busca pela bibliografia e na internet achamos duas heurísticas bastante utilizadas em problemas desse tipo.

### Distância

Baseado na real distância que o agente irá percorrer. Considera o número de passos que serão dados, o número de ações que serão tomadas.

O processamento é um pouco mais custoso do que o normal, devido a essa metodologia.



In [ ]:
    # Calcula a distancia entre duas posicoes, valido apenas para posicoes que fazem parte da sol
    @functools.lru_cache(maxsize=4096)
    def distance(self, position1, position2, lim=100000):
        if position1 == position2:
            return 0
   # dicicionario com formato { item: [distancia, visitado (0 = nao, 1 = sim)], ... }
        d = dict.fromkeys(self.reachable_positions(position1), [math.inf, 0])
        d[position1] = [0, 0]

        i = 0
        while min({l[1] for l in d.values()}) == 0 and i < lim:
            m = math.inf
            current = None
            for k, v in d.items():
                if v[1] == 0 and v[0] < m:
                    current = k
                    m = v[0]

            d[current] = [m, 1]

            neighbors = self.adjacent(current)
            for neighbor in neighbors:
                if d[neighbor][1] == 0 and m + 1 < d[neighbor][0]:
                    d[neighbor] = [m + 1, 0]
                    if neighbor == position2:
                        return m + 1

            current = None
            i = i + 1

        try:
            return d[position2][0]
        except KeyError:
            return math.inf``

O algoritmo acima calcula a **distância** entre dois pontos, se eles puderem ser atingidos um a partir do outro.
Inicialmente, ele checa se os pontos passados se referem a mesma posição, em caso positivo, já temos nossa resposta! (e nos poupa um bom processamento). Em caso negativo *distance* usa uma função auxiliar (*reachable_positions*) que nos retorna um dicionário onde dada a chave (coordenadas da respectiva posição) temos um valor (uma lista (distancia, visitado)).
Enquanto não existir elementos ainda não visitados, avaliamos o **mais próximo** dentre estes,  da posição corrente da nossa busca.

Com esse nó avaliado no loop, para cada posição adjacente não avaliada, incrementa-se a distância respectiva.

Esse processo continua até que se visite a posição dada por _position2_ 

Ou seja, em resumo, simulamos os passos a serem dados para nosso agente chegar de um ponto A (*position1*) a um ponto B (*position2*) e projetamos o menor caminho possível, sustentado pela estratégia de que iniciamos cada uma das "buscas parciais" do nó com a menor distância, nos termos de manhattan, a cada iteração.

### Distância Euclidiana


In [ ]:

def euclidean_distance(self, position1, position2):
        if self.reachable(position1, position2) is False:
            return math.inf
        
        deltaX1 = (position2[0] - position1[0])**2
        deltaY1 = (position2[1] - position1[1])**2
        return (deltaX1 + deltaY1)**0.5

Essa heurística é bem mais direta em termos de lógica e processamento de dados. 

A distância euclidiana avalia simplismente, a distância em linha reta, do ponto A ao ponto B, que no caso de nosso algoritmo de busca seriam o ponto de avaliação atual ao goal.

Em primeiro lugar, checamos se os dois pontos são alcançáveis. Se sim, calculamos a distância segundo a fórmula de Euclides:

$\sqrt{(x2 - x1)² + (y2 - y1)²)}$


## Representação do problema e seus estados

Como forma de facilitar nosso trabalho, usamos a estrutura da ferramenta AIMA, dada como auxílio para o desenvolvimento do nosso projeto. Aproveitamos a classe **__Node__** para armazenar o estado, que nada mais seria que as coordenadas (x, y) descrevendo a posição no maze.

Essa classe contém alguns métodos que nos serão muito úteis, como o método *path*, que nos dá o caminho percorrido até ali pela busca, ou o *expand* que nos retorna uma lista de nós que podemos seguir a partir das ações disponíveis.

Quanto aos métodos de busca, por estarmos aproveitando a estrutura dada também seguiu a mesma lógica do AIMA. Algumas alterações foram necessárias...

Abaixo é possível ver a definição do problema:

In [1]:
class PacmanProblem(Problem):
    def __init__(self, initial, goal, maze=None):
        self.initial = initial
        self.goal = goal
        if maze == None:
            self.maze, self.mazeY, self.mazeX = rM.readMaze()
        else:
            self.maze, self.mazeY, self.mazeX = rM.readMaze(maze)
    
    # for a given position returns the adjacent positions that are gray paths
    def adjacent(self, position):
        adjacent = []
        # left
        if self.maze[position[0], position[1]-1] == 2:
            if position[1] -1 < 0:
                adjacent.append((position[0], self.mazeX -1))
            else:
                adjacent.append((position[0], position[1]-1))
        # right
        if position[1] + 1 == self.mazeX and self.maze[position[0], 0] == 2:
            adjacent.append((position[0], 0))
        elif position[1] + 1 < self.mazeX and self.maze[position[0], position[1]+1] == 2:
            adjacent.append((position[0], position[1]+1))
                
        # up
        if self.maze[position[0]-1, position[1]] == 2:
            if position[0] -1 < 0:
                adjacent.append((self.mazeY-1, position[1]))
            else:
                adjacent.append((position[0]-1, position[1]))
        # down
        if position[0] + 1 == self.mazeY and self.maze[0, position[1]] == 2:
            adjacent.append((0, position[1]))
        elif position[0] + 1 < self.mazeY and self.maze[position[0]+1, position[1]] == 2:
            adjacent.append((position[0]+1, position[1]))
        
        return adjacent

     # returns the possible child states for the given state
    def actions(self, state):
        return self.adjacent(state)

    # the goal is reached if all positions that are reachable through the initial state are in the solution 
    # and the first and last positions in the solution are the initial position and goal position
    def goal_test(self, states):
        return states[-1] == self.goal and states[0] == self.initial
    
    def result(self, state, action):
        return action
    
    def path_cost(self, c, position1, action, position2):
        # Se as posicoes n forem adjacentes entao nao ha um caminho direto de pos1 pra pos2
        if abs(position1[0] - position2[0]) > 1 or abs(position1[1] - position2[1]) > 1:
            return math.inf
        return c+1

    # Retorna uma lista com todas as posicoes alcancaveis a partir da posicao dada, inclusive essa posicao
    def reachable_positions(self, startPosition):
        positions = []
        queue = [startPosition]

        while queue:
            position = queue.pop(0)
            positions.append(position)
            actions = self.adjacent(position)

            for action in actions:
                alreadyVisited = False
                for item in positions:
                    if item == action:
                        alreadyVisited = True
                        break

                if alreadyVisited == False:
                    queue.append(action)

        return set(positions)


    def h(self, node):
        return self.distance(node.state, self.goal)

    def g(self, node):
        return node.path_cost

NameError: name 'Problem' is not defined

Omitimos a função *distance* nesse bloco pois ela já foi mostrada anteriormente. a função adjacente retorna para uma dada posição áreas cinzas adjacentes para as quais é possível se movimentar, não criamos uma abstração do tipo 'left', 'right', 'up', 'down' pois não consideramos necessário para a resolução do problema.

Definimos o *goal_test* como: dada uma sequência de posições a primeira posição da sequência é a posição inicial do problema e a última posição é a posição de objetivo do problema, não verificamos se os itens intermediários são de fato adjacentes pois isso seria custoso nos algoritmos de busca.

A função *reachable_positions* retorna uma lista de posições que podem ser alcançadas a partir de uma posição dada como argumento.


## Breadth First Search

Abaixo segue a definição da busca em largura:

In [ ]:
def tree_breadth_search_for_vis(problem, iterLim=100000):
    """Search through the successors of a problem to find a goal.
    The argument frontier should be an empty queue.
    Don't worry about repeated paths to a state. [Figure 3.7]"""

    # we use these two variables at the time of visualisations
    iterations = 0
    all_node_colors = []
    node_colors = {k: 'white' for k in set(problem.reachable_positions(problem.initial))}

    # Adding first node to the queue
    frontier = deque([Node(problem.initial)])

    node_colors[Node(problem.initial).state] = "orange"
    iterations += 1
    all_node_colors.append(dict(node_colors))

    explored = []
    while frontier:
        # Popping first node of queue
        node = frontier.popleft()
        explored.append(node.state)
        # modify the currently searching node to red
        node_colors[node.state] = "red"
        iterations += 1
        all_node_colors.append(dict(node_colors))

        if problem.goal_test([node.state for node in node.path()]):
            # modify goal node to green after reaching the goal
            node_colors[node.state] = "green"
            iterations += 1
            all_node_colors.append(dict(node_colors))
            return (iterations, all_node_colors, node)

        for n in node.expand(problem):
            if n.state not in explored and n not in frontier:
                frontier.append(n)
                node_colors[n.state] = "orange"
                iterations += 1
                all_node_colors.append(dict(node_colors))

        # modify the color of explored nodes to gray
        node_colors[node.state] = "gray"
        iterations += 1
        all_node_colors.append(dict(node_colors))

    return None

## Depth First Search

Abaixo segue a definição da busca em profundidade:

In [ ]:
def tree_depth_search_for_vis(problem, iterLim=100000):
    """Search through the successors of a problem to find a goal.
    The argument frontier should be an empty queue.
    Don't worry about repeated paths to a state. [Figure 3.7]"""

    # we use these two variables at the time of visualisations
    iterations = 0
    all_node_colors = []
    node_colors = {k : 'white' for k in set(problem.reachable_positions(problem.initial))}

    # Adding first node to the stack
    frontier = [Node(problem.initial)]

    node_colors[Node(problem.initial).state] = "orange"
    iterations += 1
    all_node_colors.append(dict(node_colors))

    explored = []
    while len(frontier) > 0 and iterations < iterLim:
        # Popping first node of stack
        node = frontier.pop()

        explored.append(node.state)
        # modify the currently searching node to red
        node_colors[node.state] = "red"
        iterations += 1
        all_node_colors.append(dict(node_colors))

        if problem.goal_test([node.state for node in node.path()]):
            # modify goal node to green after reaching the goal
            node_colors[node.state] = "green"
            iterations += 1
            all_node_colors.append(dict(node_colors))
            return (iterations, all_node_colors, node)

        for n in node.expand(problem):
            if n.state not in explored and n not in frontier:
                frontier.append(n)
                node_colors[n.state] = "orange"
                iterations += 1
                all_node_colors.append(dict(node_colors))
            

        # modify the color of explored nodes to gray
        node_colors[node.state] = "gray"
        iterations += 1
        all_node_colors.append(dict(node_colors))

    return (iterations, all_node_colors, node)

## A* Search

Abaixo segue a definição da heurística A*, o método usa também outro método de busca chamado *best_first_graph_search_for_vis*, que será exibido depois:

In [ ]:
def astar_search_graph(problem, h=None, g=None):
    if h is None:
        h = problem.h
    if g is None:
        g = problem.g
    iterations, all_node_colors, node = best_first_graph_search_for_vis(problem, lambda n: g(n) + h(n))
    return iterations, all_node_colors, node

## Greedy Best First Search

Segue a definição da busca gulosa:

In [ ]:
def greedy_best_first_search(problem, h=None):
    if h == None:
        h = problem.h
    iterations, all_node_colors, node = best_first_graph_search_for_vis(problem, lambda n: h(n))
    return (iterations, all_node_colors, node)

## Goal Test

Simplesmente é passado o path contendo o novo node a ser avaliado e, como estrutura do problema, nosso goal.
Caso o goal esteja contido no path achamos nosso objetivo! O agente então poderá agir.

Aqui é importante lembrar que nosso agente realiza somente buscas offline, ou seja, antes de realizar uma ação, a busca é executada e uma solução é encontrada.

### 1. Breadth first search

**Completo:** Sim.

**Ótimo:** Não.

**Complexidade de tempo:** Toda vez que é feita uma expansão até 3 novos nós são adicionados à fila de prioridade, portanto **O**($3^{d+1}$), onde d+1 é a profundidade da solução mais rasa (mais próxima do estado inicial).

**Complexidade de memória:** O algoritmo armazena os nós expandidos em uma fila de prioridade e os visitados em uma lista, poranto **O**($3^{d+1}$), igual à complexidade de tempo.

**Resultados:**

*Problema1:* Tempo de execução: 0.01 segundos, iterações: 136, posições distintas visitadas: 14

*Problema2:* Tempo de execução: 0.01 segundos, iterações: 159, posições distintas visitadas: 23

*Problema3:* Tempo de execução: 0.003 segundos, iterações: 66, posições distintas visitadas: 14

### 2. Depth first search

**Completo:** Sim.

**Ótimo:** Não.

**Complexidade de tempo:** Toda vez que é feita uma expansão até 3 novos nós são adicionados à fila de prioridade, portanto **O**($3^{d+1}$), onde d+1 é a profundidade da solução mais rasa (mais próxima do estado inicial).

**Complexidade de memória:** O algoritmo armazena os nós expandidos em uma fila de prioridade e os visitados em uma lista, poranto **O**($3^{d+1}$), igual à complexidade de tempo.

**Resultados:**

*Problema1:* Tempo de execução: 0.009 segundos, iterações: 317, posições distintas visitadas: 24

*Problema2:* Tempo de execução: 0.01 segundos, iterações: 348, posições distintas visitadas: 27

*Problema3:* Tempo de execução: 0.002 segundos, iterações: 94, posições distintas visitadas: 18

### 3. Greedy Best first search

O _Greedy Best First Search Algoritm_ funciona de maneira muito parecida com o _A* Search_. A partir de um nó inicial, avalia-se o custo f(n) do caminho a cada um de seus possíveis destinos, de acordo com as ações que poderão ser tomadas.

Onde,

  f(n) = g(n) + h(n) 
 
  g(n) é o custo do caminho tomado e h(n) é o custo segundo a heurística adotada para a avaliação.


Seu nome greedy (ganancioso) se dá pelo fato de que após uma ação tomada, não faz mais parte da avaliação (até que se comprove que o caminho não se dá até o goal) os custos perantes as demais ações possíveis, os outros nós adjacentes no nosso caso.

<p align="center">
  <img width="400" height="400" src="Greedy.png">
</p>

Acima vemos uma imagem que ilustra o comportamento do algoritmo, vemos claramente a preferência pelo caminho mais barato ali naquele momento, sem levar em consideração outros passos.

**Completo:** Não.

**Ótimo:** Não.

**Complexidade de tempo:** O(b^m). 

**Complexidade de memória:** O(b^m).

Em ambos b é a quantidade de nós que a atual posição pode expandir e m refere-se à profundidade da busca.
Mas é importante ressaltar que em alguns casos, a situação pode favorecer e se tornar um algoritmo bem eficiente, como vimos através dos resultados obtidos.
O grande problema da greedy solution é o fato de não ter controle em situações de espaço infinito, ou em caminhos sem solução, por exemplo com loops (que acabarão por "prender" nosso agente num ciclo infinito). Outra situação ruim é que nem sempre optará pelo caminho menos custoso, o que no nosso caso não chegou a ser um problema, mas em situações de maior escala podem vir a ser um grande problema.

**Resultados:**

*Problema1:* Tempo de execução: 0.003 segundos, iterações: 47, posições distintas visitadas: 14

*Problema2:* Tempo de execução: 0.004 segundos, iterações: 77, posições distintas visitadas: 23

*Problema3:* Tempo de execução: 0.001 segundos, iterações: 47, posições distintas visitadas: 14

### 4. A*

**Completo:** Sim.

**Ótimo:** Não.

**Complexidade de tempo:** Toda vez que é feita uma expansão até 3 novos nós são adicionados à fila de prioridade, portanto **O**($3^{d+1}$), onde d+1 é a profundidade da solução mais rasa (mais próxima do estado inicial).

**Complexidade de memória:** O algoritmo armazena os nós expandidos em uma fila de prioridade e os visitados em uma lista, poranto **O**($3^{d+1}$), igual à complexidade de tempo.

**Resultados:**

*Problema1:* Tempo de execução: 0.13 segundos, iterações: 156, posições distintas visitadas: 18

*Problema2:* Tempo de execução: 0.21 segundos, iterações: 182, posições distintas visitadas: 23

*Problema3:* Tempo de execução: 0.02 segundos, iterações: 47, posições distintas visitadas: 14

### 5. Hill climbing search

Foi escolhido este método de busca local para encontrar mínimos da função distância (a distância entre o nó atual e o nó destino). Espera-se que o único mínimo encontrado seja o próprio nó destino pois é o ponto em que a função tem o seu mínimo global (0), além de que não é esperado mais nenhum mínimo local.

**Completo:** Não. Foram encontrados casos em que o algoritmo se entrou em um caminho sem saída e determinou que este era o mínimo (o objetivo) pois não podia sair de lá, sendo que ele chegou a este lugar pois era a posição mais próxima ao nó destino dentre os vizinhos do nó anterior.

**Ótimo:** Não

**Complexidade de tempo:** No melhor dos casos o algoritmo implementado atravessaria uma reta até o objetivo se este existir, no pior dos casos ele pode precisar se mover pelo tabuleiro todo, sendo este delimitado apenas pelas circunstâncias do tabuleiro, devido à natureza do problema.

**Complexidade de memória:** O algoritmo armazena todos os nós expandidos a partir do nó atual, para este problema, o número de nós adjacentes é limitado à 8, logo são armazenados até no máximo 9 nós ao mesmo tempo.

**Resultados:**

*Problema1:* Tempo de execução: 0.004 segundos, iterações: 173, posições distintas visitadas: 18

*Problema2:* Tempo de execução: 0.007 segundos, iterações: 182, posições distintas visitadas: 23

*Problema3:* Tempo de execução: 0.003 segundos, iterações: 71, posições distintas visitadas: 14

Abaixo se encontra o código utilizado para o método de busca local com alguns ajustes para podermos realizar a visualização dos passos utilizando a função `show_map` de *notebook.py*.

In [3]:
from local_search import hill_climbing_search
from notebook import psource
psource(hill_climbing_search)